<div style="text-align: center; font-size: 30px; font-weight: bold; margin-bottom: 20px;">
    Program 2
</div>


### **Aim**
Understanding MLP Hyperparameter Tuning using GridSearch and RandomSearch. Perform hyperparameter tuning for a Multilayer Perceptron using GridSearch and RandomSearch using credit-approval and MNIST Dataset

### **Theory**

#### Multilayer Perceptron (MLP)

A Multilayer Perceptron is a fully connected feedforward neural network consisting of an input layer, one or more hidden layers, and an output layer. Each neuron computes a weighted sum of inputs followed by an activation function, enabling the network to learn non-linear mappings. MLPs are trained using backpropagation, where gradients of the loss function propagate backward to update weights.

Key hyperparameters that influence MLP performance include the number of hidden layers, number of neurons per layer, activation functions, learning rate, regularization strength, optimizer choice, and batch size.

#### Hyperparameter Tuning

Hyperparameters are settings that control how the model learns but are not learned directly from the data. Choosing good hyperparameters is essential for optimal accuracy, stability, and convergence.

#### Grid Search

GridSearchCV performs an exhaustive search over all possible combinations of specified hyperparameters. For each combination, the model is trained using cross-validation, and the configuration that yields the best validation score is selected.
Characteristics:

* Tests every combination
* Very reliable for small search spaces
* Computationally expensive as the search space grows

#### Random Search

RandomizedSearchCV selects random combinations of hyperparameters from specified distributions. Instead of checking all possibilities, it evaluates a fixed number of random samples.
Characteristics:

* Much faster and more efficient for large search spaces
* Often finds high-performing configurations with far fewer trials
* Probabilistic approach that provides flexibility

#### Importance of Tuning

Hyperparameter tuning helps avoid underfitting and overfitting, improves model generalization, and significantly boosts accuracy. Grid search provides systematic exploration, while random search offers efficient sampling of large hyperparameter spaces. Both methods help identify the best MLP configuration for a dataset.

### **Source Code**

#### Credit Approval Dataset

In [6]:
import numpy as np
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score

In [ ]:
data = fetch_openml("credit-g", version=1, as_frame=True)
X = data.data
y = (data.target == "good").astype(int)

numeric_cols = X.select_dtypes(include=["int64", "float64"]).columns
categorical_cols = X.select_dtypes(include=["object", "category"]).columns

# Preprocessing pipeline
preprocess = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), numeric_cols),
        ("cat", OneHotEncoder(handle_unknown="ignore"), categorical_cols),
    ]
)

# Split dataset
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)


In [10]:
# Training pipeline

pipeline = Pipeline([
    ("preprocess", preprocess),
    ("mlp", MLPClassifier(max_iter=1500, random_state=42))
])


#### Grid Search Implementation

In [11]:
# Grid params
param_grid = {
    "mlp__hidden_layer_sizes": [(50,), (100,), (64, 32)],
    "mlp__activation": ["relu", "tanh"],
    "mlp__alpha": [0.0001, 0.001, 0.01],
    "mlp__learning_rate": ["constant", "adaptive"],
}

In [12]:
grid_search = GridSearchCV(
    pipeline,
    param_grid,
    cv=3,
    n_jobs=-1
)

grid_search.fit(X_train, y_train)

print("Best GridSearch Parameters:")
print(grid_search.best_params_)


Best GridSearch Parameters:
{'mlp__activation': 'relu', 'mlp__alpha': 0.001, 'mlp__hidden_layer_sizes': (50,), 'mlp__learning_rate': 'constant'}


In [13]:
y_pred = grid_search.predict(X_test)
acc = accuracy_score(y_test, y_pred)

print("GridSearch Test Accuracy:", acc)

GridSearch Test Accuracy: 0.79


#### Random Search

In [14]:
param_dist = {
    "mlp__hidden_layer_sizes": [(50,), (100,), (128, 64), (64, 32, 16)],
    "mlp__activation": ["relu", "tanh", "logistic"],
    "mlp__alpha": np.logspace(-4, -1, 10),
    "mlp__learning_rate_init": np.logspace(-4, -2, 10),
    "mlp__solver": ["adam", "sgd"]
}

In [17]:
random_search = RandomizedSearchCV(
    pipeline,
    param_dist,
    n_iter=15,
    cv=3,
    n_jobs=-1,
    random_state=42
)

random_search.fit(X_train, y_train)

print("Best RandomSearch Parameters:")
print(random_search.best_params_)

Best RandomSearch Parameters:
{'mlp__solver': 'adam', 'mlp__learning_rate_init': np.float64(0.001291549665014884), 'mlp__hidden_layer_sizes': (50,), 'mlp__alpha': np.float64(0.002154434690031882), 'mlp__activation': 'relu'}


In [18]:
y_pred = random_search.predict(X_test)
acc = accuracy_score(y_test, y_pred)

print("RandomSearch Test Accuracy:", acc)

RandomSearch Test Accuracy: 0.805


#### MNIST Dataset

In [19]:
mnist = fetch_openml("mnist_784", version=1)
X = mnist.data.astype("float32")
y = mnist.target.astype("int")

X /= 255.0  # scale pixels 0–1

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.1, random_state=42
)

In [22]:
mnist_pipe = Pipeline([
    ("scaler", StandardScaler()),
    ("mlp", MLPClassifier(max_iter=200, random_state=42))
])

In [23]:
param_grid_mnist = {
    "mlp__hidden_layer_sizes": [(64,), (128,), (256,)],
    "mlp__activation": ["relu", "tanh"],
    "mlp__alpha": [0.0001, 0.001]
}

grid_mnist = GridSearchCV(
    mnist_pipe,
    param_grid_mnist,
    cv=2,
    n_jobs=-1
)

grid_mnist.fit(X_train[:2000], y_train[:2000])

print("Best MNIST GridSearch Parameters:")
print(grid_mnist.best_params_)

Best MNIST GridSearch Parameters:
{'mlp__activation': 'relu', 'mlp__alpha': 0.001, 'mlp__hidden_layer_sizes': (256,)}


In [24]:
y_pred = grid_mnist.predict(X_test[:2000])
acc = accuracy_score(y_test[:2000], y_pred)

print("MNIST GridSearch Accuracy:", acc)

MNIST GridSearch Accuracy: 0.9035


#### RandomSearch

In [25]:
param_dist_mnist = {
    "mlp__hidden_layer_sizes": [(128,), (256,), (128, 64)],
    "mlp__activation": ["relu", "tanh"],
    "mlp__alpha": np.logspace(-4, -2, 5),
    "mlp__learning_rate_init": np.logspace(-4, -2, 5),
}

rand_mnist = RandomizedSearchCV(
    mnist_pipe,
    param_dist_mnist,
    n_iter=8,
    cv=2,
    n_jobs=-1,
    random_state=42
)

rand_mnist.fit(X_train[:2000], y_train[:2000])

print("Best MNIST RandomSearch Parameters:")
print(rand_mnist.best_params_)

Best MNIST RandomSearch Parameters:
{'mlp__learning_rate_init': np.float64(0.0031622776601683794), 'mlp__hidden_layer_sizes': (128, 64), 'mlp__alpha': np.float64(0.01), 'mlp__activation': 'relu'}


In [26]:
y_pred = rand_mnist.predict(X_test[:2000])
acc = accuracy_score(y_test[:2000], y_pred)

print("MNIST RandomSearch Accuracy:", acc)


MNIST RandomSearch Accuracy: 0.904
